In [7]:
pip install jax

     ---------------------------------------- 1.7/1.7 MB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: C:\Users\Abdulla\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import numpy as np
import jax
import jax.numpy as jnp

class BasePlanarQuadrotor:

    def __init__(self):
        # Dynamics constants
        # yapf: disable
        self.x_dim = 6         # state dimension (see dynamics below)
        self.u_dim = 2         # control dimension (see dynamics below)
        self.g = 9.807         # gravity (m / s**2)
        self.m = 2.5           # mass (kg)
        self.l = 1.0           # half-length (m)
        self.Iyy = 1.0         # moment of inertia about the out-of-plane axis (kg * m**2)
        self.Cd_v = 0.25       # translational drag coefficient
        self.Cd_phi = 0.02255  # rotational drag coefficient
        # yapf: enable

        # Control constraints
        self.max_thrust_per_prop = 0.75 * self.m * self.g  # total thrust-to-weight ratio = 1.5
        self.min_thrust_per_prop = 0  # at least until variable-pitch quadrotors become mainstream :D

    def ode(self, state, control, np=jnp):
        """Continuous-time dynamics of a planar quadrotor expressed as an ODE."""
        x, v_x, y, v_y, phi, omega = state
        T_1, T_2 = control
        return np.array([
            v_x,
            (-(T_1 + T_2) * np.sin(phi) - self.Cd_v * v_x) / self.m,
            v_y,
            ((T_1 + T_2) * np.cos(phi) - self.Cd_v * v_y) / self.m - self.g,
            omega,
            ((T_2 - T_1) * self.l - self.Cd_phi * omega) / self.Iyy,
        ])

    def discrete_step(self, state, control, dt, np=jnp):
        """Discrete-time dynamics (Euler-integrated) of a planar quadrotor."""
        # RK4 would be more accurate, but this runs more quickly in a homework problem;
        # in this notebook we use Euler integration for both control and simulation for
        # illustrative purposes (i.e., so that planning and simulation match exactly).
        # Often simulation may use higher fidelity models than those used for planning/
        # control, e.g., using `scipy.integrate.odeint` here for much more accurate
        # (and expensive) integration.
        return state + dt * self.ode(state, control, np)

    def A(self, state, control, dt):
        """Jacobian matrix for the discrete-time dynamics."""
        x, v_x, y, v_y, phi, omega = state
        T_1, T_2 = control
        return np.array([
            [1, dt, 0, 0, 0, 0],
            [0, -self.Cd_v*dt/self.m + 1, 0, 0, dt*(-T_1 - T_2)*np.cos(phi)/self.m, 0],
            [0, 0, 1, dt, 0, 0],
            [0, 0, 0, -self.Cd_v*dt/self.m + 1, -dt*(T_1 + T_2)*np.sin(phi)/self.m, 0],
            [0, 0, 0, 0, 1, dt],
            [0, 0, 0, 0, 0, -self.Cd_phi*dt/self.Iyy + 1]
        ]) # type: ignore

    def B(self, state, control, dt):
        """Jacobian matrix for the discrete-time control input."""
        x, v_x, y, v_y, phi, omega = state
        T_1, T_2 = control
        return np.array([
            [0, 0],
            [-dt*np.sin(phi)/self.m, -dt*np.sin(phi)/self.m],
            [0, 0],
            [dt*np.cos(phi)/self.m, dt*np.cos(phi)/self.m],
            [0, 0],
            [-dt*self.l/self.Iyy, dt*self.l/self.Iyy]
        ])
    def H(self):
      """Jacobian matrix for the measurement model."""
      return np.array([
          [1, 0, 0, 0, 0, 0], # x
          [0, 0, 1, 0, 0, 0], # y
          [0 ,0, 0 ,0, 1, 0]  # phi
      ])

In [9]:
#@title Definition of `PlanarQuadrotor`, which adds code for creating animations on top of `BasePlanarQuadrotor` above.

import matplotlib
import matplotlib.animation
import matplotlib.pyplot as plt
from IPython.core.display import HTML


class PlanarQuadrotor(BasePlanarQuadrotor):

    def animate(self, states, dt, ax=None):
        x, y, phi = states[:, 0], states[:, 2], states[:, 4]

        # Geometry
        frame_width = 2 * self.l
        frame_height = 0.15
        axle_height = 0.2
        axle_width = 0.05
        prop_width = 0.5 * frame_width
        prop_height = 1.5 * frame_height
        hub_width = 0.3 * frame_width
        hub_height = 2.5 * frame_height

        # Figure and axis
        if ax is None:
            fig, ax = plt.subplots(figsize=(12, 6))
        else:
            fig = ax.figure
        x_min, x_max = np.min(x), np.max(x)
        x_pad = (frame_width + prop_width) / 2 + 0.1 * (x_max - x_min)
        y_min, y_max = np.min(y), np.max(y)
        y_pad = (frame_width + prop_width) / 2 + 0.1 * (y_max - y_min)
        ax.set_xlim([x_min - x_pad, x_max + x_pad])
        ax.set_ylim([y_min - y_pad, y_max + y_pad])
        ax.set_aspect(1.)

        # Artists
        frame = matplotlib.patches.Rectangle((-frame_width / 2, -frame_height / 2),
                                             frame_width,
                                             frame_height,
                                             facecolor="tab:blue",
                                             edgecolor="k")
        hub = matplotlib.patches.FancyBboxPatch((-hub_width / 2, -hub_height / 2),
                                                hub_width,
                                                hub_height,
                                                facecolor="tab:blue",
                                                edgecolor="k",
                                                boxstyle="Round,pad=0.,rounding_size=0.05")
        axle_left = matplotlib.patches.Rectangle((-frame_width / 2, frame_height / 2),
                                                 axle_width,
                                                 axle_height,
                                                 facecolor="tab:blue",
                                                 edgecolor="k")
        axle_right = matplotlib.patches.Rectangle((frame_width / 2 - axle_width, frame_height / 2),
                                                  axle_width,
                                                  axle_height,
                                                  facecolor="tab:blue",
                                                  edgecolor="k")
        prop_left = matplotlib.patches.Ellipse(((axle_width - frame_width) / 2, frame_height / 2 + axle_height),
                                               prop_width,
                                               prop_height,
                                               facecolor="tab:gray",
                                               edgecolor="k",
                                               alpha=0.7)
        prop_right = matplotlib.patches.Ellipse(((frame_width - axle_width) / 2, frame_height / 2 + axle_height),
                                                prop_width,
                                                prop_height,
                                                facecolor="tab:gray",
                                                edgecolor="k",
                                                alpha=0.7)
        bubble = matplotlib.patches.Circle((0, 0), 1.5 * self.l, facecolor="None", edgecolor="red", linestyle="--")
        patches = (frame, hub, axle_left, axle_right, prop_left, prop_right, bubble)
        for patch in patches:
            ax.add_patch(patch)
        trace = ax.plot([], [], "--", linewidth=2, color="tab:orange")[0]
        timestamp = ax.text(0.1, 0.9, "", transform=ax.transAxes)

        def animate(k):
            transform = matplotlib.transforms.Affine2D().rotate_around(0., 0., phi[k])
            transform += matplotlib.transforms.Affine2D().translate(x[k], y[k])
            transform += ax.transData
            for patch in patches:
                patch.set_transform(transform)
            trace.set_data(x[:k + 1], y[:k + 1])
            timestamp.set_text("t = {:.1f} s".format(dt * k))
            artists = patches + (trace, timestamp)
            return artists

        ani = matplotlib.animation.FuncAnimation(fig, animate, len(states), interval=dt * 1000, blit=True)
        plt.close(fig)
        return HTML(ani.to_html5_video())

In [10]:
#@title Implementation of a direct method for computing a nominal trajectory driving from stable hover at $(x, y) = (0, 5)$ to $(10, 7)$, avoiding an obstacle centered at $(5, 5)$.

import scipy.optimize


def optimize_nominal_trajectory(N=50, return_optimize_result=False):
    planar_quad = PlanarQuadrotor()
    x_0 = np.array([0., 0., 5., 0., 0., 0.])
    x_f = np.array([10., 0., 7., 0., 0., 0.])

    equilibrium_thrust = 0.5 * planar_quad.m * planar_quad.g
    x_dim = planar_quad.x_dim
    u_dim = planar_quad.u_dim

    def pack_decision_variables(final_time, states, controls):
        """Packs decision variables (final_time, states, controls) into a 1D vector.

        Args:
            final_time: scalar.
            states: array of shape (N + 1, x_dim).
            controls: array of shape (N, u_dim).
        Returns:
            An array `z` of shape (1 + (N + 1) * x_dim + N * u_dim,).
        """
        return np.concatenate([[final_time], states.ravel(), controls.ravel()])

    def unpack_decision_variables(z):
        """Unpacks a 1D vector into decision variables (final_time, states, controls).

        Args:
            z: array of shape (1 + (N + 1) * x_dim + N * u_dim,).
        Returns:
            final_time: scalar.
            states: array of shape (N + 1, x_dim).
            controls: array of shape (N, u_dim).
        """
        final_time = z[0]
        states = z[1:1 + (N + 1) * x_dim].reshape(N + 1, x_dim)
        controls = z[-N * u_dim:].reshape(N, u_dim)
        return final_time, states, controls

    def cost(z):
        final_time, states, controls = unpack_decision_variables(z)
        dt = final_time / N
        return final_time + dt * np.sum(np.square(controls - equilibrium_thrust))

    z_guess = pack_decision_variables(10, x_0 + np.linspace(0, 1, N + 1)[:, np.newaxis] * (x_f - x_0),
                                      equilibrium_thrust * np.ones((N, u_dim)))

    bounds = scipy.optimize.Bounds(
        pack_decision_variables(0., -np.inf * np.ones((N + 1, x_dim)),
                                planar_quad.min_thrust_per_prop * np.ones((N, u_dim))),
        pack_decision_variables(np.inf, np.inf * np.ones((N + 1, x_dim)),
                                planar_quad.max_thrust_per_prop * np.ones((N, u_dim))))

    def equality_constraints(z):
        final_time, states, controls = unpack_decision_variables(z)
        dt = final_time / N
        constraint_list = [states[i + 1] - planar_quad.discrete_step(states[i], controls[i], dt, np) for i in range(N)]
        constraint_list.append(states[0] - x_0)
        constraint_list.append(states[-1] - x_f)
        return np.concatenate(constraint_list)

    def inequality_constraints(z):
        final_time, states, controls = unpack_decision_variables(z)
        return np.sum(np.square(states[:, [0, 2]] - np.array([5, 5])), -1) - 3**2

    result = scipy.optimize.minimize(cost,
                                     z_guess,
                                     bounds=bounds,
                                     constraints=[{
                                         "type": "eq",
                                         "fun": equality_constraints
                                     }, {
                                         "type": "ineq",
                                         "fun": inequality_constraints
                                     }])
    if return_optimize_result:
        return result
    return unpack_decision_variables(result.x)


def plot_obstacle(ax):
    obstacle = matplotlib.patches.Circle((5, 5), 3 - 1.5 * planar_quad.l, facecolor="k")
    ax.add_patch(obstacle)

In [11]:
planar_quad = PlanarQuadrotor()

final_time, nominal_states, nominal_controls = optimize_nominal_trajectory()
initial_state = nominal_states[0]
N = len(nominal_controls)
dt = final_time / N

In [ ]:
def simulate_nominal(initial_state, nominal_controls):
    states = [initial_state]
    for k in range(N):
        states.append(planar_quad.discrete_step(states[k], nominal_controls[k], dt))
    return np.array(states)  # Equals `nominal_states` from optimization.


fig, ax = plt.subplots(figsize=(12, 6))
plot_obstacle(ax)
planar_quad.animate(simulate_nominal(initial_state, nominal_controls), dt, ax)

In [ ]:
#@title Wind field code, including definition of `apply_wind_disturbance`.

np.random.seed(1)
xs, ys = np.arange(-3, 15), np.arange(3, 10)
X, Y = np.meshgrid(xs, ys, indexing="ij")
XY = np.stack([X.ravel(), Y.ravel()], -1)
Wx, Wy = np.random.multivariate_normal(np.zeros(len(XY)),
                                       np.exp(-np.sum(np.square(XY[:, None] - XY[None, :]), -1) / 16), 2)
wind = scipy.interpolate.RegularGridInterpolator((xs, ys),
                                                 np.stack([Wx, Wy], -1).reshape(X.shape + (2,)),
                                                 bounds_error=False,
                                                 fill_value=0)


def apply_wind_disturbance(state, dt):
    x, v_x, y, v_y, phi, omega = state
    dv_x, dv_y = dt * wind(np.array([x, y]))[0]
    return np.array([x, v_x + dv_x, y, v_y + dv_y, phi, omega])


def plot_nominal_trajectory(ax):
    ax.plot(*nominal_states[:, [0, 2]].T)


def plot_wind(ax):
    ax.quiver(X, Y, Wx, Wy, width=3e-3, alpha=0.2)

In [ ]:
def simulate_open_loop(initial_state, nominal_controls):
    states = [initial_state]
    for k in range(N):
        next_state = planar_quad.discrete_step(states[k], nominal_controls[k], dt)
        next_state = apply_wind_disturbance(next_state, dt)
        states.append(next_state)
    return np.array(states)


planar_quad = PlanarQuadrotor()
fig, ax = plt.subplots(figsize=(12, 6))
plot_obstacle(ax)
plot_nominal_trajectory(ax)
plot_wind(ax)
planar_quad.animate(simulate_open_loop(initial_state, nominal_controls), dt, ax)

In [ ]:
Q = 1e3 * np.diag([1, 1e-2, 1, 1e-2, 1e-2, 1e-2])
R = np.diag([1, 1])

In [ ]:
### Add some code here ###

def linearize_dynamics_autodifferentiation(dynamics_func, state0, control0,dt) -> tuple:
  '''
  dynamics_func: a function that takes in two inputs, state and control. i.e., dynamics_func(state, control)
  HINT: read documentation for jac.jacobian
  '''
  A = jax.jacfwd(dynamics_func,argnums=0)(state0,control0,dt)
  B = jax.jacfwd(dynamics_func,argnums=1)(state0,control0,dt)
  C = dynamics_func(state0,control0,dt) -A@state0 -B@control0
  return A, B, C

nominal_states = simulate_nominal(initial_state,nominal_controls)

A_list = []
B_list = []
for i in range(len(nominal_states)-1): #50 points only in nominal_control
  A, B, C = linearize_dynamics_autodifferentiation(planar_quad.discrete_step,nominal_states[i],nominal_controls[i],dt)
#   A, B, C = linearize_dynamics_autodifferentiation(planar_quad.ode,nominal_states[i],nominal_controls[i])
  A_list.append(A)
  B_list.append(B)


def discrete_lqr(Q,R,N,A_list,B_list):
  P = [None] * (N + 1)
  K = [None] * N

  #Initialize P_T = Q_T
  P[-1] = Q

  #backward in time to find Ps
  for i in range(N,0,-1):
  #specify linearized A and B matrix
    A = A_list[i-1]
    B = B_list[i-1]
    P[i-1] = Q + (A.T @ P[i] @ A) - (A.T @ P[i] @ B) @ jnp.linalg.pinv(R+B.T @ P[i] @ B) @ (B.T @ P[i] @ A)

  #foward to find Ks
  for i in range(N):
    K[i] = -jnp.linalg.pinv(R+B.T@P[i+1]@B) @ B.T @ P[i+1] @ A

  return K

#####################

# K_solution = np.load("K_solution.npy")

def simulate_closed_loop(initial_state, nominal_controls, nominal_states):
  states = [initial_state]
  K_sequence = discrete_lqr(Q,R,N,A_list,B_list) #50 Ks
  for k in range(N):
#     print(f"{k}th error:",K_sequence[k]-K_solution[k])
    delta_x = states[k]-nominal_states[k]
    # delta_u = K_solution[k] @ delta_x
    delta_u = K_sequence[k] @ delta_x


    control = nominal_controls[k]+delta_u

    control = np.clip(control, planar_quad.min_thrust_per_prop, planar_quad.max_thrust_per_prop)
    next_state = planar_quad.discrete_step(states[k], control, dt)
    next_state = apply_wind_disturbance(next_state, dt)
    states.append(next_state)
  return np.array(states)


planar_quad = PlanarQuadrotor()
fig, ax = plt.subplots(figsize=(12, 6))
plot_obstacle(ax)
plot_nominal_trajectory(ax)
plot_wind(ax)
planar_quad.animate(simulate_closed_loop(initial_state, nominal_controls,nominal_states), dt, ax)

In [ ]:
### Add some code here ###

def linearize_dynamics_autodifferentiation(dynamics_func, state0, control0,dt) -> tuple:
  '''
  dynamics_func: a function that takes in two inputs, state and control. i.e., dynamics_func(state, control)
  HINT: read documentation for jac.jacobian
  '''
  A = jax.jacfwd(dynamics_func,argnums=0)(state0,control0,dt)
  B = jax.jacfwd(dynamics_func,argnums=1)(state0,control0,dt)
  C = dynamics_func(state0,control0,dt) -A@state0 -B@control0
  return A, B, C

nominal_states = simulate_nominal(initial_state,nominal_controls)

A_list = []
B_list = []
for i in range(len(nominal_states)-1): #50 points only in nominal_control
  A, B, C = linearize_dynamics_autodifferentiation(planar_quad.discrete_step,nominal_states[i],nominal_controls[i],dt)
#   A, B, C = linearize_dynamics_autodifferentiation(planar_quad.ode,nominal_states[i],nominal_controls[i])
  A_list.append(A)
  B_list.append(B)


def discrete_lqr(Q,R,N,A_list,B_list):
  P = [None] * (N + 1)
  K = [None] * N

  #Initialize P_T = Q_T
  P[-1] = Q

  #backward in time to find Ps
  for i in range(N,0,-1):
  #specify linearized A and B matrix
    A = A_list[i-1]
    B = B_list[i-1]
    P[i-1] = Q + (A.T @ P[i] @ A) - (A.T @ P[i] @ B) @ jnp.linalg.pinv(R+B.T @ P[i] @ B) @ (B.T @ P[i] @ A)

  #foward to find Ks
  for i in range(N):
    K[i] = -jnp.linalg.pinv(R+B.T@P[i+1]@B) @ B.T @ P[i+1] @ A

  return K

#####################

# K_solution = np.load("K_solution.npy")

def simulate_closed_loop(initial_state, nominal_controls, nominal_states):
  states = [initial_state]
  K_sequence = discrete_lqr(Q,R,N,A_list,B_list) #50 Ks
  for k in range(N):
#     print(f"{k}th error:",K_sequence[k]-K_solution[k])
    delta_x = states[k]-nominal_states[k]
    # delta_u = K_solution[k] @ delta_x
    delta_u = K_sequence[k] @ delta_x


    control = nominal_controls[k]+delta_u

    control = np.clip(control, planar_quad.min_thrust_per_prop, planar_quad.max_thrust_per_prop)
    next_state = planar_quad.discrete_step(states[k], control, dt)
    # next_state = apply_wind_disturbance(next_state, dt)
    states.append(next_state)
  return np.array(states)


planar_quad = PlanarQuadrotor()
fig, ax = plt.subplots(figsize=(12, 6))
plot_obstacle(ax)
plot_nominal_trajectory(ax)
# plot_wind(ax)
planar_quad.animate(simulate_closed_loop(initial_state, nominal_controls,nominal_states), dt, ax)

In [ ]:
### Add some code here ###

def linearize_dynamics_autodifferentiation(dynamics_func, state0, control0,dt) -> tuple:
  '''
  dynamics_func: a function that takes in two inputs, state and control. i.e., dynamics_func(state, control)
  HINT: read documentation for jac.jacobian
  '''
  A = jax.jacfwd(dynamics_func,argnums=0)(state0,control0,dt)
  B = jax.jacfwd(dynamics_func,argnums=1)(state0,control0,dt)
  C = dynamics_func(state0,control0,dt) -A@state0 -B@control0
  return A, B, C

nominal_states = simulate_nominal(initial_state,nominal_controls)

A_list = []
B_list = []
for i in range(len(nominal_states)-1): #50 points only in nominal_control
  A, B, C = linearize_dynamics_autodifferentiation(planar_quad.discrete_step,nominal_states[i],nominal_controls[i],dt)
#   A, B, C = linearize_dynamics_autodifferentiation(planar_quad.ode,nominal_states[i],nominal_controls[i])
  A_list.append(A)
  B_list.append(B)


def discrete_lqr(Q,R,N,A_list,B_list):
  P = [None] * (N + 1)
  K = [None] * N

  #Initialize P_T = Q_T
  P[-1] = Q

  #backward in time to find Ps
  for i in range(N,0,-1):
  #specify linearized A and B matrix
    A = A_list[i-1]
    B = B_list[i-1]
    P[i-1] = Q + (A.T @ P[i] @ A) - (A.T @ P[i] @ B) @ jnp.linalg.pinv(R+B.T @ P[i] @ B) @ (B.T @ P[i] @ A)

  #foward to find Ks
  for i in range(N):
    K[i] = -jnp.linalg.pinv(R+B.T@P[i+1]@B) @ B.T @ P[i+1] @ A

  return K

#####################


def simulate_closed_loop(initial_state, nominal_controls, nominal_states):
  states = [initial_state]
  K_sequence = discrete_lqr(Q,R,N,A_list,B_list) #50 Ks
    # covariance matrix of noise
  q = np.eye(6) * 0.01
  r = np.eye(3) * 0.01

  # gaussian noise
  # v = np.sqrt(q) @ np.random.normal(0,1,6)
  v = np.sqrt(q) @ np.random.normal(0,2,6)
  # w = np.sqrt(r) @ np.random.normal(0,0.5,3)
  w = np.sqrt(r) @ np.random.normal(0,1.5,3)

  # true trajectory = ideal state + noise
  nominal_states = nominal_states + v

  for k in range(N):
#     print(f"{k}th error:",K_sequence[k]-K_solution[k])
    delta_x = states[k]-nominal_states[k]
    delta_u = K_sequence[k] @ delta_x


    control = nominal_controls[k]+delta_u

    control = np.clip(control, planar_quad.min_thrust_per_prop, planar_quad.max_thrust_per_prop)
    next_state = planar_quad.discrete_step(states[k], control, dt)
    # next_state = apply_wind_disturbance(next_state, dt)
    states.append(next_state)
  return np.array(states)


planar_quad = PlanarQuadrotor()
fig, ax = plt.subplots(figsize=(12, 6))
plot_obstacle(ax)
plot_nominal_trajectory(ax)
# plot_wind(ax)
planar_quad.animate(simulate_closed_loop(initial_state, nominal_controls,nominal_states), dt, ax)

In [ ]:
def EKF(dym,action,mu,state,sigma,dt,A,B):
    # H = dym.H()
    # Q = np.eye(6) * 0.01
    # R = np.eye(3) * 0.01

    # m, n = H.shape
    # # print(H.shape)

    # v = np.sqrt(Q) @ np.random.normal(0,1,n)
    # # w = np.sqrt(R) @ np.random.normal(0,0.5,m)

    # # true trajectory = ideal state + noise
    # true_traj = state + v

    # ada = np.array([[1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0]])

    # Y = ada @ true_traj

    # #prediction
    # Xhat_pred = dym.discrete_step(mu, action, dt)

    # X_next = np.array(Xhat_pred) + 0.1 * np.array(Y - ada @ Xhat_pred)
    # print(mu, state)
    # return X_next
    # A = dym.A(mu, action, dt)
    # B = dym.B(mu, action, dt)
    H = dym.H()
    m, n = H.shape

    # covariance matrix of noise
    Q = np.eye(n) * 0.01
    R = np.eye(m) * 0.01

    # gaussian noise
    v = np.sqrt(Q) @ np.random.normal(0,0.6,n)
    w = np.sqrt(R) @ np.random.normal(0,0.5,m)

    # true trajectory = ideal state + noise
    true_traj = state + v

    # measurement = true trajectory + noise
    Y = H @ true_traj
    print("Y", Y)

    # time update
    # Xhat_pred = A @ mu + B @ action # predicted state
    Xhat_pred = dym.discrete_step(mu, action, dt)
    P_pred = A @ sigma @ A.T + Q # predicted covariance

    print("og", state)
    print("pred", Xhat_pred)

    #S = H @ sigma @ H.T + R
    S = H @ P_pred @ H.T + R
    S_inv = np.linalg.inv(S)
    K = P_pred @ H.T @ S_inv # Kalman gain

    # information update
    X_next = Xhat_pred + K @ (Y - H @ Xhat_pred)
    P_next = P_pred - K @ S @ K.T

    print("est", X_next)

    return X_next, P_next, Y, H @ true_traj
    #return state, P_next, Y, H @ true_traj

In [ ]:
### Add some code here ###

def linearize_dynamics_autodifferentiation(dynamics_func, state0, control0,dt) -> tuple:
  '''
  dynamics_func: a function that takes in two inputs, state and control. i.e., dynamics_func(state, control)
  HINT: read documentation for jac.jacobian
  '''
  A = jax.jacfwd(dynamics_func,argnums=0)(state0,control0,dt)
  B = jax.jacfwd(dynamics_func,argnums=1)(state0,control0,dt)
  C = dynamics_func(state0,control0,dt) -A@state0 -B@control0
  return A, B, C

nominal_states = simulate_nominal(initial_state,nominal_controls)

A_list = []
B_list = []
for i in range(len(nominal_states)-1): #50 points only in nominal_control
  A, B, C = linearize_dynamics_autodifferentiation(planar_quad.discrete_step,nominal_states[i],nominal_controls[i],dt)
#   A, B, C = linearize_dynamics_autodifferentiation(planar_quad.ode,nominal_states[i],nominal_controls[i])
  A_list.append(A)
  B_list.append(B)


def discrete_lqr(Q,R,N,A_list,B_list):
  P = [None] * (N + 1)
  K = [None] * N

  #Initialize P_T = Q_T
  P[-1] = Q

  #backward in time to find Ps
  for i in range(N,0,-1):
  #specify linearized A and B matrix
    A = A_list[i-1]
    B = B_list[i-1]
    P[i-1] = Q + (A.T @ P[i] @ A) - (A.T @ P[i] @ B) @ jnp.linalg.pinv(R+B.T @ P[i] @ B) @ (B.T @ P[i] @ A)

  #foward to find Ks
  for i in range(N):
    K[i] = -jnp.linalg.pinv(R+B.T@P[i+1]@B) @ B.T @ P[i+1] @ A

  return K

#####################


def simulate_closed_loop(initial_state, nominal_controls, nominal_states):
  states = [initial_state]
  K_sequence = discrete_lqr(Q,R,N,A_list,B_list) #50 Ks

  dynamics = BasePlanarQuadrotor()
  x0 = np.array([0., 0., 5., 0., 0., 0.])
  x0_ekf = np.array([0., 0., 5., 0., 0., 0.])
  sigma_0 = np.eye(6) * 0.01
  ideal_states = [initial_state]

  for i in range(N):
      # ideal path
      action = nominal_controls[i]
      x_next = dynamics.discrete_step(x0,action,dt)
      # x_next = nominal_states[i]
      x0 = x_next
      nominal_states[i+1] = x_next
      ideal_states.append(x_next)

      # ekf
      # ekf_next = EKF(dynamics, action, x0_ekf, x_next, dt)
      ekf_next, sigma_next, Y, true_traj = EKF(dynamics, action, x0_ekf, x_next, sigma_0, dt, A_list[i], B_list[i])
      x0_ekf = ekf_next
      sigma_0 = sigma_next
      nominal_states[i+1] = ekf_next

  # for i in range(len(ideal_states)):
  #   print(ideal_states[i], nominal_states[i])

  for k in range(N):
#     print(f"{k}th error:",K_sequence[k]-K_solution[k])
    delta_x = states[k]-nominal_states[k]
    delta_u = K_sequence[k] @ delta_x


    control = nominal_controls[k]+delta_u

    control = np.clip(control, planar_quad.min_thrust_per_prop, planar_quad.max_thrust_per_prop)
    next_state = planar_quad.discrete_step(states[k], control, dt)
    # next_state = apply_wind_disturbance(next_state, dt)
    states.append(next_state)
  return np.array(states)


planar_quad = PlanarQuadrotor()
fig, ax = plt.subplots(figsize=(12, 6))
plot_obstacle(ax)
plot_nominal_trajectory(ax)
# plot_wind(ax)
planar_quad.animate(simulate_closed_loop(initial_state, nominal_controls,nominal_states), dt, ax)

Y [0.01325857 4.9635444  0.05360692]
og [0.         0.         5.         0.09578753 0.         0.00570384]
pred [0.         0.         5.         0.09578753 0.         0.00570384]
est [ 0.00888156 -0.03058577  4.9755793   0.09376829  0.03590982  0.00871301]
Y [-0.02118183  5.0413895   0.01489637]
og [0.0000000e+00 0.0000000e+00 5.0163503e+00 1.8622962e-01 9.7361516e-04
 9.7340029e-03]
pred [ 3.6607282e-03 -9.3469664e-02  4.9915848e+00  1.8310636e-01
  3.7397087e-02  1.2731592e-02]
est [-0.01203768 -0.08683531  5.023251    0.19004919  0.02341032  0.0092306 ]
Y [-0.02664183  5.124599    0.0595625 ]
og [ 0.0000000e+00 -1.7158070e-03  5.0481386e+00  2.7135280e-01
  2.6351577e-03  1.2631098e-02]
pred [-0.02686001 -0.12660544  5.0556912   0.27462506  0.02498593  0.01212963]
est [-0.02718317 -0.14741214  5.099687    0.28988346  0.0470551   0.02013623]
Y [ 0.04845148  5.1078053  -0.03374051]
og [-2.9287912e-04 -6.3203471e-03  5.0944571e+00  3.5117742e-01
  4.7912188e-03  1.4713388e-02]
pred [

In [ ]:
def EKF(dym,action,mu,state,sigma,dt,A,B):
    # H = dym.H()
    # Q = np.eye(6) * 0.01
    # R = np.eye(3) * 0.01

    # m, n = H.shape
    # # print(H.shape)

    # v = np.sqrt(Q) @ np.random.normal(0,1,n)
    # # w = np.sqrt(R) @ np.random.normal(0,0.5,m)

    # # true trajectory = ideal state + noise
    # true_traj = state + v

    # ada = np.array([[1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0]])

    # Y = ada @ true_traj

    # #prediction
    # Xhat_pred = dym.discrete_step(mu, action, dt)

    # X_next = np.array(Xhat_pred) + 0.1 * np.array(Y - ada @ Xhat_pred)
    # print(mu, state)
    # return X_next
    # A = dym.A(mu, action, dt)
    # B = dym.B(mu, action, dt)
    H = dym.H()
    m, n = H.shape

    # covariance matrix of noise
    Q = np.eye(n) * 0.01
    R = np.eye(m) * 0.01

    # gaussian noise
    v = np.sqrt(Q) @ np.random.normal(0,0.5,n)
    w = np.sqrt(R) @ np.random.normal(0,0.5,m)

    # true trajectory = ideal state + noise
    true_traj = state + v

    # measurement = true trajectory + noise
    Y = H @ true_traj
    print("Y", Y)

    # time update
    # Xhat_pred = A @ mu + B @ action # predicted state
    Xhat_pred = dym.discrete_step(mu, action, dt)
    P_pred = A @ sigma @ A.T #+ Q # predicted covariance

    print("og", state)
    print("pred", Xhat_pred)

    #S = H @ sigma @ H.T + R
    S = H @ P_pred @ H.T + R
    S_inv = np.linalg.inv(S)
    K = P_pred @ H.T @ S_inv # Kalman gain

    # information update
    X_next = Xhat_pred + K @ (Y - H @ Xhat_pred)
    #P_next = P_pred - K @ S @ K.T
    P_next  = (np.eye(6) - K @ H) @ P_pred

    print("est", X_next)

    return X_next, P_next, Y, H @ true_traj
    #return state, P_next, Y, H @ true_traj

In [ ]:
### Add some code here ###

def linearize_dynamics_autodifferentiation(dynamics_func, state0, control0,dt) -> tuple:
  '''
  dynamics_func: a function that takes in two inputs, state and control. i.e., dynamics_func(state, control)
  HINT: read documentation for jac.jacobian
  '''
  A = jax.jacfwd(dynamics_func,argnums=0)(state0,control0,dt)
  B = jax.jacfwd(dynamics_func,argnums=1)(state0,control0,dt)
  C = dynamics_func(state0,control0,dt) -A@state0 -B@control0
  return A, B, C

nominal_states = simulate_nominal(initial_state,nominal_controls)

A_list = []
B_list = []
for i in range(len(nominal_states)-1): #50 points only in nominal_control
  A, B, C = linearize_dynamics_autodifferentiation(planar_quad.discrete_step,nominal_states[i],nominal_controls[i],dt)
#   A, B, C = linearize_dynamics_autodifferentiation(planar_quad.ode,nominal_states[i],nominal_controls[i])
  A_list.append(A)
  B_list.append(B)


def discrete_lqr(Q,R,N,A_list,B_list):
  P = [None] * (N + 1)
  K = [None] * N

  #Initialize P_T = Q_T
  P[-1] = Q

  #backward in time to find Ps
  for i in range(N,0,-1):
  #specify linearized A and B matrix
    A = A_list[i-1]
    B = B_list[i-1]
    P[i-1] = Q + (A.T @ P[i] @ A) - (A.T @ P[i] @ B) @ jnp.linalg.pinv(R+B.T @ P[i] @ B) @ (B.T @ P[i] @ A)

  #foward to find Ks
  for i in range(N):
    K[i] = -jnp.linalg.pinv(R+B.T@P[i+1]@B) @ B.T @ P[i+1] @ A

  return K

#####################


def simulate_closed_loop(initial_state, nominal_controls, nominal_states):
  states = [initial_state]
  K_sequence = discrete_lqr(Q,R,N,A_list,B_list) #50 Ks

  dynamics = BasePlanarQuadrotor()
  x0 = np.array([0., 0., 5., 0., 0., 0.])
  x0_ekf = np.array([0., 0., 5., 0., 0., 0.])
  sigma_0 = np.eye(6) * 0.01
  ideal_states = [initial_state]

  for i in range(N):
      # ideal path
      action = nominal_controls[i]
      x_next = dynamics.discrete_step(x0,action,dt)
      # x_next = nominal_states[i]
      x0 = x_next
      nominal_states[i+1] = x_next
      ideal_states.append(x_next)

      # ekf
      # ekf_next = EKF(dynamics, action, x0_ekf, x_next, dt)
      ekf_next, sigma_next, Y, true_traj = EKF(dynamics, action, x0_ekf, x_next, sigma_0, dt, A_list[i], B_list[i])
      x0_ekf = ekf_next
      sigma_0 = sigma_next
      nominal_states[i+1] = ekf_next

  # for i in range(len(ideal_states)):
  #   print(ideal_states[i], nominal_states[i])

  for k in range(N):
#     print(f"{k}th error:",K_sequence[k]-K_solution[k])
    delta_x = states[k]-nominal_states[k]
    delta_u = K_sequence[k] @ delta_x


    control = nominal_controls[k]+delta_u

    control = np.clip(control, planar_quad.min_thrust_per_prop, planar_quad.max_thrust_per_prop)
    next_state = planar_quad.discrete_step(states[k], control, dt)
    # next_state = apply_wind_disturbance(next_state, dt)
    states.append(next_state)
  return np.array(states)


planar_quad = PlanarQuadrotor()
fig, ax = plt.subplots(figsize=(12, 6))
plot_obstacle(ax)
plot_nominal_trajectory(ax)
# plot_wind(ax)
planar_quad.animate(simulate_closed_loop(initial_state, nominal_controls,nominal_states), dt, ax)

Y [ 0.02360921  4.979711   -0.05367549]
og [0.         0.         5.         0.09578753 0.         0.00570384]
pred [0.         0.         5.         0.09578753 0.         0.00570384]
est [ 1.1974108e-02  4.8767310e-02  4.9897099e+00  9.4109923e-02
 -2.7223110e-02  1.2059403e-03]
Y [0.06057053 5.003415   0.00614528]
og [0.0000000e+00 0.0000000e+00 5.0163503e+00 1.8622962e-01 9.7361516e-04
 9.7340029e-03]
pred [ 0.02029843  0.09600712  5.005774    0.18392642 -0.02701726  0.00525342]
est [ 0.0334186   0.07535674  5.0049243   0.18356265 -0.01756366  0.01186023]
Y [-0.02758137  5.121551    0.03685796]
og [ 0.0000000e+00 -1.7158070e-03  5.0481386e+00  2.7135280e-01
  2.6351577e-03  1.2631098e-02]
pred [ 0.04628159  0.10502139  5.0362573   0.2684602  -0.01553919  0.01474914]
est [ 0.01311759 -0.01173204  5.062263    0.28619242  0.00694235  0.02129594]
Y [-0.04614316  5.154795    0.0102459 ]
og [-2.9287912e-04 -6.3203471e-03  5.0944571e+00  3.5117742e-01
  4.7912188e-03  1.4713388e-02]
pred [